# Desarrollo del Modelo Predictivo Definitivo
En este notebook se implementa y entrena el modelo definitivo para la predicción de la compensación total anual (CompTotal). A partir de los datos previamente filtrados y procesados, se aplicaron los siguientes pasos clave:

#### 1) Filtrado y Transformación de Datos:
- Se establecieron límites razonables para la variable objetivo (CompTotal), filtrando valores entre 18,000 y 55,000 euros.
- Para reducir la variabilidad y manejar la asimetría positiva de los datos, se aplicó una transformación logarítmica a la variable objetivo.

#### 2) Selección de Características:
- Se utilizaron 35 variables seleccionadas previamente como las más relevantes según el proceso de feature engineering.

#### 3) Normalización:
- Se escaló el conjunto de características utilizando MinMaxScaler para garantizar que todas las variables estuvieran en el mismo rango y facilitar el entrenamiento del modelo.
#### 4) Modelos y Ensamble:
- Se definieron tres modelos base: Random Forest Regressor, Gradient Boosting Regressor y XGBoost Regressor.
- Estos modelos se combinaron mediante un Voting Regressor con pesos ajustados (2.8, 0.6 y 1.5 respectivamente) para optimizar su contribución al ensamble.
#### 5) Validación Cruzada y Evaluación:
- Se utilizó validación cruzada con 5 folds para evaluar la robustez del modelo. Las métricas promedio de validación incluyeron R², MAE, MSE y MAPE, proporcionando una visión clara del rendimiento del modelo.
- Finalmente, el modelo se evaluó en un conjunto de prueba independiente, obteniendo métricas que reflejan su capacidad de generalización.
#### 6) Exportación del Modelo:
- El modelo final y el escalador fueron exportados en formato pickle, quedando listos para su implementación en el pipeline de predicción.
Este notebook establece la base para la predicción de nuevas observaciones y representa el paso final en el desarrollo del modelo dentro del proyecto. El modelo será probado con datos externos en el siguiente notebook para validar su rendimiento en escenarios reales.

In [4]:
# Importar las librerías necesarias
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
import xgboost as xgb
import joblib
import os

# Cargar el DataFrame desde el archivo pickle
with open('../Pickles/df_final.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

# Mostrar todas las columnas del DataFrame para inspección
pd.set_option('display.max_columns', None)

# Filtrar el DataFrame según límites razonables de la variable objetivo (CompTotal)
limite_inferior = 18000
limite_superior = 55000
df_filtrado = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]
df = df_filtrado

# Aplicar una transformación logarítmica a la variable objetivo para reducir la variabilidad
df['CompTotal'] = np.log1p(df['CompTotal'])

# Separar la variable objetivo (y) del resto de las características
y = df['CompTotal']
df = df.drop(columns=['CompTotal'], axis=1)

# Seleccionar las columnas más relevantes para el modelo
df = df[['YearsCodePro', 'LearnCodeOnline_encoded', 'DevType_encoded', 'LearnCode_encoded', 'CodingActivities_encoded', 
         'DatabaseHaveWorkedWith_Redis', 'ToolsTechHaveWorkedWith_Docker', 'ToolsTechHaveWorkedWith_Ant', 'YearsCode', 
         'ToolsTechHaveWorkedWith_Homebrew', 'ToolsTechHaveWorkedWith_Kubernetes', 'LanguageWantToWorkWith_GDScript', 
         'LanguageHaveWorkedWith_PHP', 'DatabaseHaveWorkedWith_IBM DB2', 'DatabaseHaveWorkedWith_MySQL', 'EdLevel', 
         'LanguageWantToWorkWith_OCaml', 'is_full_time', 'ToolsTechHaveWorkedWith_Google Test', 'LanguageWantToWorkWith_Kotlin', 
         'ToolsTechHaveWorkedWith_APT', 'LanguageWantToWorkWith_PHP', 'AISent', 'LanguageWantToWorkWith_Lua', 
         'LanguageHaveWorkedWith_Kotlin', 'DatabaseHaveWorkedWith_PostgreSQL', 'Industry_Category_Salud y Educación', 
         'LanguageHaveWorkedWith_Ruby', 'LanguageWantToWorkWith_Visual Basic (.Net)', 'DatabaseHaveWorkedWith_Oracle', 
         'LanguageWantToWorkWith_Swift', 'Frequency_2', 'Frequency_1', 'LanguageHaveWorkedWith_MATLAB', 'LanguageWantToWorkWith_C']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# Escalar los datos para normalizar las características
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Definir los modelos base
random_forest = RandomForestRegressor(random_state=42)
xgb_reg = xgb.XGBRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)

# Crear un modelo Voting Regressor con pesos ajustados
weights = [2.8, 0.6, 1.5]  # Pesos asignados a los modelos base
voting_regressor = VotingRegressor(
    estimators=[
        ('rf', random_forest), 
        ('xgb', xgb_reg), 
        ('gb', gb_model)
    ],
    weights=weights
)

# Ajustar el Voting Regressor al conjunto de entrenamiento
voting_regressor.fit(X_train, y_train)

# Validación cruzada con 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores, mse_scores, mae_scores, mape_scores = [], [], [], []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Entrenar el modelo en el fold
    voting_regressor.fit(X_train_fold, y_train_fold)

    # Predecir en el conjunto de validación
    y_val_pred_log = voting_regressor.predict(X_val_fold)
    y_val_pred = np.expm1(y_val_pred_log)
    y_val_actual = np.expm1(y_val_fold)

    # Calcular métricas de validación
    r2_scores.append(r2_score(y_val_actual, y_val_pred))
    mse_scores.append(mean_squared_error(y_val_actual, y_val_pred))
    mae_scores.append(mean_absolute_error(y_val_actual, y_val_pred))
    mape_scores.append(np.mean(np.abs((y_val_actual - y_val_pred) / y_val_actual)) * 100)

# Calcular métricas promedio de validación cruzada
mean_r2_cv = np.mean(r2_scores)
mean_mse_cv = np.mean(mse_scores)
mean_rmse_cv = np.sqrt(mean_mse_cv)
mean_mae_cv = np.mean(mae_scores)
mean_mape_cv = np.mean(mape_scores)

print("=== Métricas de Cross-Validation ===")
print(f"R² (promedio CV): {mean_r2_cv:.4f}")
print(f"MSE (promedio CV): {mean_mse_cv:.4f}")
print(f"RMSE (promedio CV): {mean_rmse_cv:.4f}")
print(f"MAE (promedio CV): {mean_mae_cv:.4f}")
print(f"MAPE (promedio CV): {mean_mape_cv:.4f}%")

# Predicciones en el conjunto de prueba
y_test_pred_log = voting_regressor.predict(X_test)
y_test_pred = np.expm1(y_test_pred_log)
y_test_actual = np.expm1(y_test)

# Calcular métricas en el conjunto de prueba
r2_test = r2_score(y_test_actual, y_test_pred)
mse_test = mean_squared_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
mape_test = np.mean(np.abs((y_test_actual - y_test_pred) / y_test_actual)) * 100

print("=== Métricas de Test ===")
print(f"R² (Test): {r2_test:.4f}")
print(f"MSE (Test): {mse_test:.4f}")
print(f"RMSE (Test): {rmse_test:.4f}")
print(f"MAE (Test): {mae_test:.4f}")
print(f"MAPE (Test): {mape_test:.4f}%")

=== Métricas de Cross-Validation ===
R² (promedio CV): 0.4803
MSE (promedio CV): 53938437.6244
RMSE (promedio CV): 7344.2792
MAE (promedio CV): 5612.4109
MAPE (promedio CV): 16.0197%
=== Métricas de Test ===
R² (Test): 0.5378
MSE (Test): 46292536.9682
RMSE (Test): 6803.8619
MAE (Test): 5170.7225
MAPE (Test): 14.9695%


Guardo y exporto el modelo y el escalador

In [5]:
ruta_pickles = os.path.join("..", "Pickles")
# Ruta completa para los archivos
ruta_scaler = os.path.join(ruta_pickles, 'Scaler.pkl')
ruta_modelo = os.path.join(ruta_pickles, 'MiModelo.pkl')

# Exporta el escalador entrenado
joblib.dump(scaler, ruta_scaler)
# Exporta el modelo entrenado
joblib.dump(voting_regressor, ruta_modelo)

['..\\Pickles\\MiModelo.pkl']

Una vez guardado el modelo y el escalador, en el notebook siguiente se cargara el modelo y testeara con datos de nuevas viviendas para probar su rendimiento 